# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2022-03-24	Ratio of Hydrophobic Hydrophilic and Positive Negative Residues at Lipid Water Interface Influences Surface Expression and Channel Gating of TRPV1	The Journal of Membrane Biology (2022)	This paper is about the number 1. The number 2 is left for future work.	Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).	Role of LWI residuesin channel gating of TRPV1	http://academicpages.github.io/files/paper1.pdf	http://academicpaages.github.io/files/slides1.pdf
2022-07-30	Cytochrome C interacts with the pathogenic mutational hotspot region of TRPV4 and forms complexes that differ in mutation and metal ion-sensitive manner	Biochemical and Biophysical Research Communications (BBRC)	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	Cytochrome C forms complexes with TRPV4	http://academicpages.github.io/files/pa

## Import pandas

We are using the very handy pandas library for dataframes.

In [4]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [5]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2022-03-24,Ratio of Hydrophobic Hydrophilic and Positive ...,The Journal of Membrane Biology (2022),This paper is about the number 1. The number 2...,"Your Name, You. (2009). ""Paper Title Number 1....",Role of LWI residuesin channel gating of TRPV1,http://academicpages.github.io/files/paper1.pdf,http://academicpaages.github.io/files/slides1.pdf
1,2022-07-30,Cytochrome C interacts with the pathogenic mut...,Biochemical and Biophysical Research Communica...,This paper is about the number 2. The number 3...,"Your Name, You. (2010). ""Paper Title Number 2....",Cytochrome C forms complexes with TRPV4,http://academicpages.github.io/files/paper2.pdf,http://academicpages.github.io/files/slides2.pdf
2,2022-12-20,TRPM8 as a potential target in neurodegenerati...,Alzheimer's & Dementia®: The Journal of the Al...,This paper is about the number 3. The number 4...,"Your Name, You. (2015). ""Paper Title Number 3....",TRPM8 as a potential target in neurodegenerati...,http://academicpages.github.io/files/paper3.pdf,http://academicpages.github.io/files/slides3.pdf
3,2024-07-30,Lipid water interface residues of TRPM8 have c...,Journal of Molecular Biology,This paper is about the number 3. The number 4...,"Your Name, You. (2015). ""Paper Title Number 3....",LWI residues of TRPM8 have diverse effect on h...,http://academicpages.github.io/files/paper4.pdf,http://academicpages.github.io/files/slides4.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [6]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [7]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [8]:
!ls ../_publications/

'2022-03-24-Role of LWI residuesin channel gating of TRPV1.md'
'2022-07-30-Cytochrome C forms complexes with TRPV4.md'
'2022-12-20-TRPM8 as a potential target in neurodegenerative diseases.md'
'2024-07-30-LWI residues of TRPM8 have diverse effect on healt disorders.md'


In [9]:
!cat ../_publications/2022-07-30-Cythochrome C forms complex with TRPV4.md

cat: ../_publications/2022-07-30-Cythochrome: No such file or directory
cat: C: No such file or directory
cat: forms: No such file or directory
cat: complex: No such file or directory
cat: with: No such file or directory
cat: TRPV4.md: No such file or directory
